# Évaluation de la qualité de l'air 
***
Comparaison internationale des mesures de qualité de l'air  
***

In [ ]:
import pandas as pd 
import numpy as np 
from scipy import stats 
# on va d'abord charger le csv 
df = pd.read_csv("openaq.csv", sep = ";", encoding = "UTF-8")
# on fait en sorte que tout ce qui n'est pas encodable en UTF-8 dégage 
for row in df :
  row.encode('utf-8','ignore').decode("utf-8")
display(df)
df.dtypes


1. Ne garder qu'une seule date 

In [ ]:
# on ne retient que les mesures datant du 6 août 2021
df = df.loc[df["Last Updated"].str.contains("2021-08-06")]
display(df)

2. Vérifier qu'on n'a qu'une seule mesure par polluant par ville

In [ ]:
# ne garder que des polluants uniques pour chaque ville (le plus élevé)
# max_poll = df.groupby(["City", "Pollutant"])["Value"].max()
# df = pd.merge(df, max_poll, how = "inner", on = ["Pollutant", "City", "Value"])
# duplicated_city = df.duplicated(subset = ["City"]) 
# duplicated_city_df = df[duplicated_city]
# duplicated_all = duplicated_city_df.duplicated(subset = ["Pollutant"])
# duplicated_df = duplicated_city_df[duplicated_all]
# print(duplicated_df)
max_poll = df.groupby(["City", "Pollutant"])["Value"].max()
df = pd.merge(df, max_poll, how = "inner", on = ["Pollutant", "City", "Value"])
df = df.drop_duplicates(subset = ["City", "Pollutant"])
display(df)

3. Repérer les outliers 
***

In [ ]:
# on va calculer le Z-score des différentes valeurs 
z_score = stats.zscore(df["Value"], ddof = 1, nan_policy = "omit")
# et sélectionner les valeurs pour lesquelles l'écart à la moyenne est supérieur à 3 écarts types
mask = np.abs(z_score) > 3
outliers = df[mask]
display(outliers)
outliers.to_csv("outliers.csv", sep = ";", encoding = "UTF-8", )
df_clean = df.loc[df["Value"] > 0]
df_clean.to_csv("air_quality_clean.csv", sep = ";", encoding = "UTF-8")